# Analysis 2: How Corporate Lobbying is related to Firm Size.

In [1]:
import os
import time
import pickle
from typing import List, Tuple, Union

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.special import gammaln
from joblib import Parallel, delayed
from joblib.externals.loky.process_executor import TerminatedWorkerError
import statsmodels.api as sm
from utils import open_csv, save_csv

##### Load bill position data with firm size

In [2]:
client_year_df = open_csv('analysis_input/analysis2_firm_size_bill_position_df.csv')

### Helper Functions

In [3]:
def create_design_matrix(df: pd.DataFrame, predictors: List[str], vary_var: str,
                         numeric_predictors: List[str] = None) -> Tuple[pd.DataFrame, List[str]]:
    """
    Create a design matrix from df using the list of predictors.
    
    Predictors in numeric_predictors (including vary_var) are treated as numeric;
    all others are converted to categorical dummy variables.
    An intercept column is added.
    """
    if numeric_predictors is None:
        numeric_predictors = [vary_var]
    elif vary_var not in numeric_predictors:
        numeric_predictors.append(vary_var)

    X_parts = []
    col_names = []
    for col in predictors:
        if col in numeric_predictors:
            X_parts.append(df[[col]])
            col_names.append(col)
        else:
            dummies = pd.get_dummies(df[col].astype('category'), prefix=col, drop_first=True)
            X_parts.append(dummies)
            col_names.extend(dummies.columns.tolist())
    X_df = pd.concat(X_parts, axis=1)
    X_df.insert(0, 'Intercept', 1)
    return X_df, ['Intercept'] + col_names



def dirichlet_neg_log_likelihood(theta: np.ndarray, X: np.ndarray, Y: np.ndarray,
                                 k: int, p: int) -> float:
    """
    Negative log-likelihood for the Dirichlet regression model.
    """
    beta = np.array(theta).reshape((k, p))
    X_arr = np.array(X, dtype=np.float64)
    Y_arr = np.array(Y, dtype=np.float64)
    XB = np.dot(X_arr, beta.T)
    alpha = np.exp(XB)
    
    if np.any(~np.isfinite(alpha)):
        return np.inf
        
    sum_alpha = np.sum(alpha, axis=1)
    ll = np.sum(gammaln(sum_alpha)) - np.sum(gammaln(alpha)) + np.sum((alpha - 1) * np.log(Y_arr))
    return -ll


def bootstrap_iteration(args: tuple) -> np.ndarray:
    """
    One bootstrap replicate for the Dirichlet regression QOI.
    Returns delta_bs computed from the bootstrap sample.
    """
    np.random.seed()

    (df_dir, predictors, response_cols, cluster_var, initial_theta, k, p, num_clusters,
     numeric_predictors, std_log_emp_10, std_log_emp_90, design_cols, vary_var, full_data) = args

    sampled_clusters = np.random.choice(df_dir[cluster_var].unique(), size=num_clusters, replace=True)
    boot_sample = pd.concat([df_dir[df_dir[cluster_var] == clust] for clust in sampled_clusters])
    
    X_boot_df, _ = create_design_matrix(boot_sample, predictors, vary_var, numeric_predictors=numeric_predictors)
    X_boot_df = X_boot_df.reindex(columns=design_cols, fill_value=0).astype(float)
    X_boot = X_boot_df.values
    Y_boot = boot_sample[response_cols].values

    try:
        result_boot = minimize(dirichlet_neg_log_likelihood, initial_theta,
                               args=(X_boot, Y_boot, k, p),
                               method='L-BFGS-B',
                               options={'maxfun': 50000, 'maxiter': 30000})
        beta_boot = result_boot.x.reshape((k, p))
    except Exception as e:
        print(f"Bootstrap error: {e}")
        return np.full(k, np.nan)
    
    if np.isnan(beta_boot).any():
        return np.full(k, np.nan)

    # Replace key predictor with standardized 10th and 90th values.
    idx = design_cols.index(vary_var)
    X_boot_10 = X_boot.copy()
    X_boot_90 = X_boot.copy()
    X_boot_10[:, idx] = std_log_emp_10
    X_boot_90[:, idx] = std_log_emp_90

    XB_10 = np.matmul(X_boot_10, beta_boot.T)
    A10 = np.exp(XB_10)
    props_10 = A10 / np.sum(A10, axis=1, keepdims=True)
    avg_props_10 = np.mean(props_10, axis=0)

    XB_90 = np.matmul(X_boot_90, beta_boot.T)
    A90 = np.exp(XB_90)
    props_90 = A90 / np.sum(A90, axis=1, keepdims=True)
    avg_props_90 = np.mean(props_90, axis=0)

    delta_bs = avg_props_90 - avg_props_10
    return delta_bs


def bootstrap_iteration_with_retry(args: tuple, max_retries: int = 3) -> np.ndarray:
    """
    Wrapper for bootstrap_iteration that retries if an error occurs.
    """
    for attempt in range(max_retries):
        try:
            return bootstrap_iteration(args)
        except (TerminatedWorkerError, MemoryError, FloatingPointError, ValueError) as e:
            print(f"Error '{e}' on attempt {attempt + 1}; retrying...")
            time.sleep(2)
    (df_dir, _, _, _, _, k, _, num_clusters,
     _, _, _, _, vary_var, full_data) = args
    return np.full(k, np.nan)

### Main Function: run_models_and_plot

In [4]:
def run_models_and_plot(data: pd.DataFrame, predictors: List[str], vary_var: str,
                        response_cols: List[str], logistic_response: str, cluster_var: str,
                        num_bootstrap: int = 100, numeric_predictors: List[str] = None) -> Union[dict, None]:
    """
    Runs Dirichlet and logistic regressions.
    Generates a figure and CSV summaries for the QOI.
    """
    start_time = time.time()

    if numeric_predictors is None:
        numeric_predictors = [vary_var]
    elif vary_var not in numeric_predictors:
        numeric_predictors.append(vary_var)

    # --- Prepare Dirichlet Data ---
    dirichlet_cols = [cluster_var] + predictors + response_cols
    df_dir = data[dirichlet_cols].dropna().copy()
    for pred in numeric_predictors:
        if pred in df_dir.columns:
            mean_val = df_dir[pred].mean()
            std_val = df_dir[pred].std()
            if std_val > 0:
                df_dir[pred] = (df_dir[pred] - mean_val) / std_val
            print(f"{pred} in Dirichlet subset: mean={mean_val:.4f}, std={std_val:.4f}")

    epsilon = 1e-6
    df_dir[response_cols] = df_dir[response_cols] + epsilon
    df_dir[response_cols] = df_dir[response_cols].div(df_dir[response_cols].sum(axis=1), axis=0)

    # --- Dirichlet Regression: Create Design Matrix ---
    X_design_df, design_cols = create_design_matrix(df_dir, predictors, vary_var, numeric_predictors=numeric_predictors)
    X_design_df = X_design_df.astype(float)
    X_design = X_design_df.values
    n, p = X_design.shape
    print(f"Dirichlet design matrix dimensions: {n} rows x {p} columns")
    Y = df_dir[response_cols].values
    k = Y.shape[1]

    np.random.seed(42)
    initial_theta = np.random.normal(0, 0.1, k * p)

    print(f"Starting Dirichlet regression with {p} parameters per category, {k} categories")
    try:
        result = minimize(dirichlet_neg_log_likelihood, initial_theta,
                          args=(X_design, Y, k, p),
                          method='L-BFGS-B',
                          options={'maxfun': 50000, 'maxiter': 30000})
        beta_hat = result.x.reshape((k, p))
        print("Dirichlet Optimization success:", result.success)
        coef_df = pd.DataFrame(beta_hat, columns=design_cols, index=[f"label{j+1}" for j in range(k)])
        print("\nDirichlet Coefficients by Category:")
        print(coef_df)
    except Exception as e:
        print(f"Error in Dirichlet regression: {e}")
        return None

    # --- Percentile Calculations for vary_var ---
    log_emp_10 = np.percentile(df_dir[vary_var].dropna(), 10)
    log_emp_90 = np.percentile(df_dir[vary_var].dropna(), 90)
    print(f"10th percentile of {vary_var} (Dirichlet subset): {log_emp_10}")
    print(f"90th percentile of {vary_var} (Dirichlet subset): {log_emp_90}")
    std_log_emp_10 = log_emp_10
    std_log_emp_90 = log_emp_90

    # --- Compute Dirichlet QOI ---
    idx = design_cols.index(vary_var)
    X_design_10 = X_design.copy()
    X_design_90 = X_design.copy()
    X_design_10[:, idx] = std_log_emp_10
    X_design_90[:, idx] = std_log_emp_90
    
    XB_10 = np.matmul(X_design_10, beta_hat.T)
    A10 = np.exp(XB_10)
    proportions_10 = A10 / np.sum(A10, axis=1, keepdims=True)
    avg_props_10 = np.mean(proportions_10, axis=0)
    
    XB_90 = np.matmul(X_design_90, beta_hat.T)
    A90 = np.exp(XB_90)
    proportions_90 = A90 / np.sum(A90, axis=1, keepdims=True)
    avg_props_90 = np.mean(proportions_90, axis=0)
    
    delta_dir = avg_props_90 - avg_props_10

    # --- Parallel Bootstrapping with Checkpointing ---
    unique_clusters = df_dir[cluster_var].unique()
    num_clusters = len(unique_clusters)
    all_boot_results = []
    batch_size = 500  # Adjust as needed
    num_batches = int(np.ceil(num_bootstrap / batch_size))

    # Define arguments for bootstrap iterations
    bs_args = (df_dir, predictors, response_cols, cluster_var,
               initial_theta, k, p, num_clusters, numeric_predictors,
               std_log_emp_10, std_log_emp_90, design_cols, vary_var, data)

    for b in range(num_batches):
        batch_dir = "analysis_output/analysis2_batch_result"
        os.makedirs(batch_dir, exist_ok=True)
        batch_filename = f"{batch_dir}/batch_results_{b+1}.pkl"
        if os.path.exists(batch_filename):
            print(f"Batch {b+1} already exists; loading results.")
            with open(batch_filename, "rb") as f:
                batch_results = pickle.load(f)
            for i, result in enumerate(batch_results):
                if np.all(np.isnan(result)):
                    print(f"Iteration {i+1} in batch {b+1} failed (NaN). Rerunning...")
                    batch_results[i] = bootstrap_iteration_with_retry(bs_args)
            with open(batch_filename, "wb") as f:
                pickle.dump(batch_results, f)
            print(f"Updated batch {b+1} results saved to {batch_filename}")
        else:
            current_batch = min(batch_size, num_bootstrap - b * batch_size)
            print(f"Starting batch {b+1}/{num_batches} with {current_batch} iterations")
            batch_results = Parallel(n_jobs=-1, verbose=5)(
                delayed(bootstrap_iteration_with_retry)(bs_args) for _ in range(current_batch)
            )
            with open(batch_filename, "wb") as f:
                pickle.dump(batch_results, f)
            print(f"Saved batch {b+1} results to {batch_filename}")
        all_boot_results.extend(batch_results)

    all_boot_results = np.array(all_boot_results)
    boot_delta = np.vstack(all_boot_results)
    se_dir = np.nanstd(boot_delta, axis=0, ddof=1)
    
    # --- Prepare Logistic Data ---
    logit_cols = [cluster_var] + predictors + [logistic_response]
    df_logit = data[logit_cols].dropna().copy()
    for pred in numeric_predictors:
        if pred in df_logit.columns:
            mean_val = df_logit[pred].mean()
            std_val = df_logit[pred].std()
            if std_val > 0:
                df_logit[pred] = (df_logit[pred] - mean_val) / std_val
            print(f"{pred} in Logistic subset: mean={mean_val:.4f}, std={std_val:.4f}")
            
    # --- Logistic Regression ---
    X_logit_df, logit_design_cols = create_design_matrix(df_logit, predictors, vary_var, numeric_predictors=numeric_predictors)
    #########
    # X_logit = X_logit_df.values
    X_logit_df = X_logit_df.astype(float)
    X_logit = X_logit_df.values
    y_logit = df_logit[logistic_response].astype(float)
    print(f"Logistic design matrix dimensions: {X_logit.shape[0]} rows x {X_logit.shape[1]} columns")
    model_logit = sm.Logit(y_logit, X_logit)
    # model_logit = sm.Logit(df_logit[logistic_response], X_logit)
    #########
    result_logit = model_logit.fit(disp=0, cov_type='cluster', cov_kwds={'groups': df_logit[cluster_var]})

    # --- Logistic QOI Evaluation (Delta Method) ---
    idx_logit = logit_design_cols.index(vary_var)
    X_logit_10 = X_logit.copy()
    X_logit_90 = X_logit.copy()
    logit_log_emp_10 = np.percentile(df_logit[vary_var].dropna(), 10)
    logit_log_emp_90 = np.percentile(df_logit[vary_var].dropna(), 90)
    X_logit_10[:, idx_logit] = logit_log_emp_10
    X_logit_90[:, idx_logit] = logit_log_emp_90
    print(f"10th percentile of {vary_var} (Logistic subset): {logit_log_emp_10:.4f}")
    print(f"90th percentile of {vary_var} (Logistic subset): {logit_log_emp_90:.4f}")
    preds_10_logit = result_logit.predict(X_logit_10)
    preds_90_logit = result_logit.predict(X_logit_90)
    p10_logit = preds_10_logit.mean()
    p90_logit = preds_90_logit.mean()
    delta_logit = p90_logit - p10_logit

    grads_10 = [p * (1 - p) * x for p, x in zip(preds_10_logit, X_logit_10)]
    avg_grad_10 = np.mean(grads_10, axis=0)
    grads_90 = [p * (1 - p) * x for p, x in zip(preds_90_logit, X_logit_90)]
    avg_grad_90 = np.mean(grads_90, axis=0)
    avg_grad_diff = avg_grad_90 - avg_grad_10
    var_logit = np.dot(avg_grad_diff, np.dot(result_logit.cov_params(), avg_grad_diff))
    se_logit = np.sqrt(var_logit)

    # --- Generate Figure and Save Results ---
    ci95_lower_logit = delta_logit - 1.96 * se_logit
    ci95_upper_logit = delta_logit + 1.96 * se_logit
    ci90_lower_logit = delta_logit - 1.645 * se_logit
    ci90_upper_logit = delta_logit + 1.645 * se_logit

    ci95_lower_dir = delta_dir - 1.96 * se_dir
    ci95_upper_dir = delta_dir + 1.96 * se_dir
    ci90_lower_dir = delta_dir - 1.645 * se_dir
    ci90_upper_dir = delta_dir + 1.645 * se_dir

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 6), sharex=True,
                                   gridspec_kw={'height_ratios': [0.8, 3]})
    ax1.errorbar(delta_logit, 0,
                 xerr=[[delta_logit - ci95_lower_logit], [ci95_upper_logit - delta_logit]],
                 fmt='o', color='black', markersize=8, linewidth=1)
    ax1.errorbar(delta_logit, 0,
                 xerr=[[delta_logit - ci90_lower_logit], [ci90_upper_logit - delta_logit]],
                 fmt='o', color='black', markersize=8, linewidth=3)
    ax1.axvline(0, color='gray', linestyle='--')
    ax1.set_yticks([0])
    ax1.set_ylim(-0.3, 0.3)
    ax1.set_yticklabels(["Lobby"], fontsize=16)
    for tick in ax1.get_yticklabels():
        tick.set_verticalalignment('center')

    outcome_labels = ['Support', 'Oppose', 'Amend', 'Monitor']
    color_dict = {
        'Support': '#01897B',
        'Oppose': '#E6447B',
        'Amend': '#F7A42F',
        'Monitor': '#A900FF'
    }
    y_positions = np.linspace(1, 2.5, num=len(delta_dir))
    for i, label in enumerate(outcome_labels):
        ax2.errorbar(delta_dir[i], y_positions[i],
                     xerr=[[delta_dir[i] - ci95_lower_dir[i]], [ci95_upper_dir[i] - delta_dir[i]]],
                     fmt='o', color=color_dict[label], markersize=8, linewidth=1)
        ax2.errorbar(delta_dir[i], y_positions[i],
                     xerr=[[delta_dir[i] - ci90_lower_dir[i]], [ci90_upper_dir[i] - delta_dir[i]]],
                     fmt='o', color=color_dict[label], markersize=8, linewidth=3)
    ax2.axvline(0, color='gray', linestyle='--')
    ax2.set_yticks(y_positions)
    ax2.set_ylim(0.7, 2.8)
    ax2.set_yticklabels(outcome_labels, fontsize=16)
    for tick in ax2.get_yticklabels():
        tick.set_verticalalignment('center')
    ax2.set_xlabel("Effect of Firm Size on Predicted Probability\n(90th - 10th Percentile)", fontsize=16, labelpad=15)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig_filename = "analysis2_fig3.pdf"
    fig.savefig(f"analysis_output/{fig_filename}")

    results = {
        "delta_dir": delta_dir,
        "se_dir": se_dir,
        "ci95_lower_dir": ci95_lower_dir,
        "ci95_upper_dir": ci95_upper_dir,
        "ci90_lower_dir": ci90_lower_dir,
        "ci90_upper_dir": ci90_upper_dir,
        "delta_logit": delta_logit,
        "se_logit": se_logit,
        "ci95_lower_logit": ci95_lower_logit,
        "ci95_upper_logit": ci95_upper_logit,
        "ci90_lower_logit": ci90_lower_logit,
        "ci90_upper_logit": ci90_upper_logit,
        "figure_filename": fig_filename
    }
    plt.close(fig)

    # --- Save CSV Summaries ---
    df_logistic_results = pd.DataFrame([{
        "delta_logit": results["delta_logit"],
        "se_logit": results["se_logit"],
        "ci95_lower_logit": results["ci95_lower_logit"],
        "ci95_upper_logit": results["ci95_upper_logit"],
        "ci90_lower_logit": results["ci90_lower_logit"],
        "ci90_upper_logit": results["ci90_upper_logit"]
    }])
    save_csv('analysis_output/analysis2_fig3_results_logistic.csv', df_logistic_results)

    dirichlet_rows = []
    for i, label in enumerate(outcome_labels):
        dirichlet_rows.append({
            "outcome": label,
            "delta_dir": results["delta_dir"][i],
            "se_dir": results["se_dir"][i],
            "ci95_lower_dir": results["ci95_lower_dir"][i],
            "ci95_upper_dir": results["ci95_upper_dir"][i],
            "ci90_lower_dir": results["ci90_lower_dir"][i],
            "ci90_upper_dir": results["ci90_upper_dir"][i]
        })
    df_dirichlet_results = pd.DataFrame(dirichlet_rows)
    save_csv('analysis_output/analysis2_fig3_results_dirichlet.csv', df_dirichlet_results)

    end_time = time.time()
    print("Total execution time: {:.2f} seconds".format(end_time - start_time))
    return results

In [5]:
results = run_models_and_plot(
    data=client_year_df,
    predictors=['log_emp', 'year'],
    vary_var='log_emp',
    response_cols=['ratio_label1', 'ratio_label2', 'ratio_label3', 'ratio_label4'],
    logistic_response='lobbied',
    cluster_var='firm_id',
    num_bootstrap=5000,
    numeric_predictors=['log_emp']
)

log_emp in Dirichlet subset: mean=9.4903, std=2.0010
Dirichlet design matrix dimensions: 3517 rows x 15 columns
Starting Dirichlet regression with 15 parameters per category, 4 categories
Dirichlet Optimization success: True

Dirichlet Coefficients by Category:
        Intercept   log_emp  year_2010  year_2011  year_2012  year_2013  \
label1  -2.464736  0.036178   0.025624   0.165421   0.123797   0.093115   
label2  -2.711395  0.020351   0.007526   0.036833   0.005423   0.024292   
label3  -2.366520  0.036530  -0.006189  -0.007177  -0.130526  -0.150930   
label4  -1.955240  0.108055  -0.018825   0.045516   0.045847   0.061896   

        year_2014  year_2015  year_2016  year_2017  year_2018  year_2019  \
label1   0.044423   0.131669   0.175535   0.095040   0.085424   0.075206   
label2   0.021026   0.013254   0.005801  -0.003307  -0.033020  -0.034181   
label3  -0.208076  -0.127704  -0.138376  -0.130432  -0.193586  -0.200254   
label4   0.203645   0.151454   0.047728   0.061779  -0.029

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   54.3s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


Saved batch 1 results to analysis_output/analysis2_batch_result/batch_results_1.pkl
Starting batch 2/10 with 500 iterations


[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   52.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


Saved batch 2 results to analysis_output/analysis2_batch_result/batch_results_2.pkl
Starting batch 3/10 with 500 iterations


...

Saved batch 8 results to analysis_output/analysis2_batch_result/batch_results_8.pkl
Starting batch 9/10 with 500 iterations


[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


Saved batch 9 results to analysis_output/analysis2_batch_result/batch_results_9.pkl
Starting batch 10/10 with 500 iterations


[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   52.5s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.1min finished


Saved batch 10 results to analysis_output/analysis2_batch_result/batch_results_10.pkl
log_emp in Logistic subset: mean=6.3576, std=2.7409
Logistic design matrix dimensions: 90905 rows x 15 columns
10th percentile of log_emp (Logistic subset): -1.3567
90th percentile of log_emp (Logistic subset): 1.3115
Total execution time: 812.79 seconds
